In [ ]:
#wikipedia web scraper

In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue'
#url = 'https://www.bls.gov/news.release/jltst.t05.htm'

In [4]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html')

In [5]:
x = list(soup.find_all('table'))
table100 = x[0]
#table100
#table100 = soup.find_all('table')[1]

In [6]:
h = table100.find_all('th')

In [7]:
headers = [head.text.strip() for head in h]
print(headers)

['Rank', 'Name', 'Industry', 'Revenue (USD millions)', 'Revenue growth', 'Employees', 'Headquarters']


In [8]:
import pandas as pd
df = pd.DataFrame(columns = headers)
df

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters


In [9]:
rs = table100.find_all('tr')

In [10]:
for row in rs[1:]:
    r_data = row.find_all('td')
    each_single_row = [data.text.strip() for data in r_data]
    print(each_single_row)
    length = len(df)
    df.loc[length] = each_single_row

['1', 'Walmart', 'Retail', '648,125', '6.0%', '2,100,000', 'Bentonville, Arkansas']
['2', 'Amazon', 'Retail and cloud computing', '574,785', '11.9%', '1,525,000', 'Seattle, Washington']
['3', 'Apple', 'Electronics industry', '383,482', '-2.8%', '161,000', 'Cupertino, California']
['4', 'UnitedHealth Group', 'Healthcare', '371,622', '14.6%', '440,000', 'Minnetonka, Minnesota']
['5', 'Berkshire Hathaway', 'Conglomerate', '364,482', '20.7%', '396,500', 'Omaha, Nebraska']
['6', 'CVS Health', 'Healthcare', '357,776', '10.9%', '259,500', 'Woonsocket, Rhode Island']
['7', 'ExxonMobil', 'Petroleum industry', '344,582', '-16.7%', '61,500', 'Spring, Texas']
['8', 'Alphabet', 'Technology and cloud computing', '307,394', '8.7%', '182,502', 'Mountain View, California']
['9', 'McKesson Corporation', 'Health', '276,711', '4.8%', '48,000', 'Irving, Texas']
['10', 'Cencora', 'Pharmacy wholesale', '262,173', '9.9%', '44,000', 'Conshohocken, Pennsylvania']
['11', 'Costco', 'Retail', '242,290', '6.8%', '3

In [11]:
df.to_csv('C:/UNISA/STA1610/WIKI_DATA.csv', index = False)

In [12]:
df[df['Rank'] == '10']

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters
9,10,Cencora,Pharmacy wholesale,"262,173",9.9%,"44,000","Conshohocken, Pennsylvania"


In [2]:
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd
site = 'https://www.amazon.co.za' #amazon.co.za
url = site + '/s?k=samsung&ref=nb_sb_noss' #Site Extension
response = requests.get(url)
soup2 = BeautifulSoup(response.content, 'html.parser')
soup = BeautifulSoup(soup2.prettify(), 'html.parser')
dontstop = True
counter = 0

headers = ['Name', 'Price', 'Rating', 'URL Link', 'Rating Count']
df = pd.DataFrame(columns = headers)

while dontstop:
    x = soup.find_all('div', class_ = "s-asin")
    count = len(x)
    for i in range(count):
        data = []
    #DATA
        x = soup.find_all('div', class_ = "s-asin")
        items = x[i]
    #NAME
        name = items.find('h2')#.text.strip()
        if name is None:
            name = 'N/A'
        else:
            name = name.text.strip()
        name = str(name)
        #print('Name: ', name)
        data.append(name)
    #PRICE
        a = items.find('span', class_ = "a-price")
        '''if a is None:
            a = '0,000.00'
        else:
            a = a.text'''
        if a is None:
            a = items.find('span', class_ = "a-color-base")
            if a is None:
                a = '0,000.00'
            else:
                a = a.text
        else:
            a = a.text
        b = re.findall(r'\d{1,3},?\d{1,3}\.\d{2}', a)
        c = b[0].replace(",", "")
        price = float(c)
        if price == 0:
            price = 'N/A'
        else:
            price = "{:.2f}".format(price)
        #print('Price: ', price)
        data.append(price)
    #RATING
        rating = items.find('span', class_ = "a-icon-alt")#.text.strip()
        if rating is None:
            rating = 'N/A'
        else:
            rating = rating.text.strip()
        rating = str(rating)
        #print('Rating: ', rating)
        data.append(rating)
        #URL
        u = items.find('a', href = True)
        if (u == None) or ('#customerReviews' in u) or ('javascript:void' in u):
            item_url = 'N/A'
        else:
            item_url = u['href']
            item_url = site + item_url
        item_url = str(item_url)
        #print('Site: ', item_url)
        data.append(item_url)
        counter = counter + 1
        #print(counter)
        #print('')
    #RATING_COUNT
        rating_c = items.find('span', class_ = "a-size-base")#.text.strip()
        if rating_c is None:
            rating_c = 'N/A'
        else:
            rating_c = rating_c.text.strip()
            try:
                rating_c = int(rating_c)
            except:
                rating_c = 'N/A'
        rating_c = str(rating_c)
        #print('Rating Count: ', rating_c)
        data.append(rating_c)
    #ABOUT
        '''item_response = requests.get(item_url)
        item_soup2 = BeautifulSoup(item_response.content, 'html.parser')
        item_soup = BeautifulSoup(item_soup2.prettify(), 'html.parser')
        about_item = ""
        item_a = item_soup.find_all('li', class_ = "a-spacing-mini")
        about_count = 0
        if item_a == None:
            about_item = 'N/A'
        else:
            for abouts in item_a:
                about_item = about_item + abouts.text.strip()
                about_count = about_count + 1
                if about_count < len(item_a):
                    about_item = about_item + "; "
        data.append(about_item)''' # TOO LONG TO RUN EACH ITEM
        length = len(df)
        df.loc[length] = data
        
    n = soup.find('a', class_ = "s-pagination-next")
    if n == None:
        dontstop = False
    else:
        nxt = n['href']
        nxt = site + nxt
        url = nxt
        response = requests.get(url)
        soup2 = BeautifulSoup(response.content, 'html.parser')
        soup = BeautifulSoup(soup2.prettify(), 'html.parser')

df

,Name,Price,Rating,URL Link,Rating Count
0,"Samsung Galaxy Fit3 Smart Watch, Grey",999.00,4.1 out of 5 stars,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...,15
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...,5
2,"Samsung Galaxy Buds FE Earbuds, Graphite",999.00,4.2 out of 5 stars,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...,9
3,Samsung Galaxy A35 128GB 5G Dual Sim Smartphon...,5264.00,5.0 out of 5 stars,https://www.amazon.co.za/Samsung-Galaxy-128GB-...,2
4,Samsung Galaxy S24 FE 128GB 5G Dual SIM Smartp...,13714.44,4.0 out of 5 stars,https://www.amazon.co.za/Samsung-Galaxy-128GB-...,1
...,...,...,...,...,...
301,Samsung Galaxy A13 Case Phone Covers 2 Piece S...,149.00,N/A,https://www.amazon.co.za/Samsung-Galaxy-Covers...,N/A
302,"Exploring Galaxy Tab S4: The Illustrated, Prac...",931.00,4.3 out of 5 stars,https://www.amazon.co.za/Exploring-Galaxy-Tab-...,N/A
303,URBAN ARMOR GEAR Scout Mobile Case for Samsung...,259.00,3.7 out of 5 stars,https://www.amazon.co.za/URBAN-ARMOR-GEAR-Sams...,8
304,PanzerGlass™ | Samsung Galaxy A35 | Screen Pro...,199.00,N/A,https://www.amazon.co.za/PanzerGlassTM-Samsung...,N/A


In [27]:
srows = df[(df['Rating Count'] == 'N/A') & (df['Rating'] != 'N/A')]
srows

,Name,Price,Rating,URL Link,Rating Count
18,Samsung Galaxy A14 4G 64GB black,2979.00,4.4 out of 5 stars,https://www.amazon.co.za/Samsung-Galaxy-A14-64...,N/A
19,Samsung Galaxy A14 4G 64GB Silver,2995.00,4.4 out of 5 stars,https://www.amazon.co.za/Samsung-Galaxy-A14-64...,N/A
28,SAMSUNG Galaxy Watch 6 40mm Bluetooth Smartwat...,4799.00,4.4 out of 5 stars,https://www.amazon.co.za/SAMSUNG-Bluetooth-Sma...,N/A
36,Samsung 980 250GB NVMe M.2 Solid State Drive,1050.00,4.6 out of 5 stars,https://www.amazon.co.za/Samsung-250GB-Solid-S...,N/A
39,Sam Galaxy A34 5G EU-128-6-5G-gy Sams Galaxy A...,5299.00,4.5 out of 5 stars,https://www.amazon.co.za/Galaxy-EU-128-6-5G-gy...,N/A
41,Samsung 870 EVO 1TB SATA III Solid State Drive,2199.00,4.6 out of 5 stars,https://www.amazon.co.za/Samsung-SATA-Solid-St...,N/A
51,My Samsung Galaxy S III,350.00,4.3 out of 5 stars,https://www.amazon.co.za/My-Samsung-Galaxy-S-I...,N/A
59,Samsung 870 EVO 500GB SATA III Solid State Drive,1373.00,4.7 out of 5 stars,https://www.amazon.co.za/Samsung-500GB-Solid-S...,N/A
81,Samsung 45W 1 Port PD Travel Adapter with Type...,799.00,4.6 out of 5 stars,https://www.amazon.co.za/Samsung-Travel-Adapte...,N/A
91,Samsung Official 45W USB-C Super Fast Charging...,795.00,4.5 out of 5 stars,https://www.amazon.co.za/Samsung-Official-USB-...,N/A


In [17]:
        rating_c = items.find('span', class_ = "a-size-base")#.text.strip()
        if rating_c is None:
            rating_c = 'N/A'
        else:
            rating_c = rating_c.text.strip()
        rating_c = str(rating_c)
        #print('Rating: ', rating)

Rating:  N/A
